In [1]:
import os

%pwd

'/Users/mitpatel/Downloads/Code/GoJo-Rika/MLOPS/Text-Summarizer/research'

In [2]:
os.chdir("../")
%pwd

'/Users/mitpatel/Downloads/Code/GoJo-Rika/MLOPS/Text-Summarizer'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [4]:
%pwd

'/Users/mitpatel/Downloads/Code/GoJo-Rika/MLOPS/Text-Summarizer'

In [5]:
from src.text_summarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.text_summarizer.utils.common import create_directories, read_yaml


In [ ]:
class ConfigurationManager:
    def __init__(self, config_filepath: str = CONFIG_FILE_PATH, params_filepath: str = PARAMS_FILE_PATH) -> None:
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            tokenizer_path=config.tokenizer_path,
            metric_file_name=config.metric_file_name,
        )

        return model_evaluation_config

In [ ]:

import evaluate
import pandas as pd
import torch
from datasets import load_from_disk
from tqdm import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer


[2025-06-25 11:36:31,963: INFO: config: PyTorch version 2.7.1 available.]


/Users/mitpatel/Downloads/Code/GoJo-Rika/MLOPS/Text-Summarizer/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig) -> None:
        self.config = config

    def generate_batch_sized_chunks(self, list_of_elements: list, batch_size: int) -> iter:
        """
        Split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements.
        """
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]
    def calculate_metric_on_test_ds(
        self,
        dataset: dict,
        metric,
        model,
        tokenizer,
        batch_size: int = 16,
        device: str = "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu",
        column_text: str = "article",
        column_summary: str = "highlights",
    ):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches, strict=False), total=len(article_batches)):
            inputs = tokenizer(
                article_batch,
                max_length=1024,
                truncation=True,
                padding="max_length",
                return_tensors="pt",
            )

            inputs = {
                k: v.to(device) for k, v in inputs.items()
            }  # Move tensors to device

            summaries = model.generate(
                input_ids=inputs["input_ids"].to(device),
                attention_mask=inputs["attention_mask"].to(device),
                length_penalty=0.8,
                num_beams=4,
                max_length=96,
            )
            """ parameter for length penalty ensures that the model does not generate sequences that are too long. """

            # Finally, we decode the generated texts,
            # Replace the  token, and add the decoded texts with the references to the metric.
            decoded_summaries = [
                tokenizer.decode(
                    s, skip_special_tokens=True, clean_up_tokenization_spaces=True,
                )
                for s in summaries
            ]

            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

            metric.add_batch(predictions=decoded_summaries, references=target_batch)

        #  Finally compute and return the ROUGE scores.
        score = metric.compute()
        return score

    def evaluate(self) -> None:
        if torch.backends.mps.is_available():
            device = "mps"
        elif torch.cuda.is_available():
            device = "cuda"
        else:
            device = "cpu"
        print(f"Using device: {device}")

        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(
            self.config.model_path,
        ).to(device)

        # Load preprocessed dataset
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        rouge_metric = evaluate.load("rouge")

        score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt["test"][0:10],
            rouge_metric,
            model_pegasus,
            tokenizer,
            batch_size=2,
            column_text="dialogue",
            column_summary="summary",
        )

        # Directly use the scores without accessing fmeasure or mid
        rouge_dict = {rn: score[rn] for rn in rouge_names}

        df = pd.DataFrame(rouge_dict, index=["pegasus"])
        df.to_csv(self.config.metric_file_name, index=False)

In [11]:
config = ConfigurationManager()
model_evaluation_config = config.get_model_evaluation_config()
model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
model_evaluation_config.evaluate()

[2025-06-25 11:47:40,749: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-06-25 11:47:40,751: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-25 11:47:40,752: INFO: common: created directory at: artifacts]
[2025-06-25 11:47:40,752: INFO: common: created directory at: artifacts/model_evaluation]
Using device: mps


100%|██████████| 5/5 [00:28<00:00,  5.80s/it]

[2025-06-25 11:48:13,260: INFO: rouge_scorer: Using default tokenizer.]
